### Load libraries

In [1]:
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import joblib
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import train_test_split

### Read file

In [2]:
clean_df = pd.read_csv('data/clean.csv')

In [3]:
#define the threshold
TS_TO_diff_threshold=5

In [4]:
#extract features and targets from dataset
filtered_df=clean_df.loc[clean_df['TS_TO_diff_minutes']<TS_TO_diff_threshold,['X1','X2','X3_0','X3_1','X3_2','X3_3','TS']]
features = filtered_df[['X1','X2','X3_0','X3_1','X3_2','X3_3']].values
targets = filtered_df.TS.values

### train the model

In [5]:
#define the dl regression model
def regress_model():
    model = keras.Sequential([
    layers.Dense(6, activation='relu', input_dim=6, kernel_initializer='normal'),
    layers.Dense(10, activation='relu'),
    layers.Dense(6, activation='relu'),
    layers.Dense(1, kernel_initializer='normal')
    ])
    optimizer = tf.keras.optimizers.RMSprop(0.001)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])
    return model

In [8]:
#cross-validation
estimator = KerasRegressor(build_fn=regress_model, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=3)
results = cross_val_score(estimator, features, targets, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Results: -59281.04 (465.51) MSE


### Retrain and save the model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.33, random_state=42)
dl_model = regress_model()
dl_model.fit(X_train, y_train,epochs=100, validation_split = 0.2, verbose=0)
dl_model.save('model/dl_model') 